In [ ]:
from pathlib import Path
import json
import pyarrow
import math
import re
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import h5py

from scipy.stats import gaussian_kde
from scipy.ndimage import label
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import holoviews as hv
from holoviews import opts

hv.extension("bokeh")

from shiny import App, Inputs, Outputs, Session, reactive, render, req, ui

from utils_behavior import (
    Sleap_utils,
    HoloviewsTemplates,
    Utils,
    Processing,
    Ballpushing_utils,
    Seaborn_Templates,
)

import Config

import importlib

In [ ]:
# Short contacts

Transformed = pd.read_feather("/mnt/upramdya_data/MD/MultiMazeRecorder/Datasets/Skeleton_TNT/240115_short_contacts_no_cutoff_Data/Transformed_contacts_nocutoff.feather")

In [ ]:
Transformed = Config.cleanup_data(Transformed)

Transformed = Config.map_split_registry(Transformed)

In [ ]:
registries = Config.prepare_registries(Transformed)

In [ ]:
Transformed["identifier"] = Transformed["fly"]+str(Transformed["contact_index"])

# Contact rates

In [ ]:
importlib.reload(Config)

In [ ]:
Test = Config.get_subset_data(Transformed, value="R78A01 (ExR1 (EB))")

In [ ]:
# Define the time window (e.g., 10 seconds)
time_window = 100

# Determine the maximum time in the dataset
max_time = Test['start'].max()

# Create bins for the time windows
bins = np.arange(0, max_time + time_window, time_window)

# Create a new column 'time_window' that indicates the time window each 'start' time belongs to
Test['time_window'] = pd.cut(Test['start'], bins=bins, right=False, labels=bins[:-1])

# Group by 'time_window', 'Brain region', and 'fly' and count the number of unique contacts in each time window
contact_rate = Test.groupby(['time_window', 'Brain region', 'fly'])['identifier'].nunique().reset_index()

# Rename the columns for clarity
contact_rate.columns = ['time_window', 'Brain region', 'fly', 'contact_count']

# Plot the contact rate over time with confidence intervals
plt.figure(figsize=(12, 6))
sns.lineplot(data=contact_rate, x='time_window', y='contact_count', hue='Brain region', marker='o', palette=Config.color_dict, ci='sd')
plt.xlabel('Time Window (s)')
plt.ylabel('Number of Unique Contacts')
plt.title(f'Contact Rate Over Time for test vs control')
plt.show()

In [ ]:
# Make a rugplot of the start times for the random Nickname

# Plot the subset data

plt.figure(figsize=(10, 6))

# Create the rugplot
sns.kdeplot(data=Test, x='start', hue='Brain region', palette=Config.color_dict)

sns.ecdfplot(data=Test, x='start', hue='Brain region', palette=Config.color_dict)

# set x axis limits to time window

plt.xlim(0, 3600)

# Set the title for the figure

plt.title(f'Start Times for test vs control')

# Set the axis labels

plt.xlabel('Start time')

plt.ylabel('Nickname')

# Show the plot

plt.show()

In [ ]:

# Assuming 'data' is your dataset
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 10))

# KDE plot
sns.kdeplot(data=Test, x="start", ax=ax1, hue="Brain region", palette=Config.color_dict)
ax1.set_title("KDE Plot")

# adjust the x-axis limits
ax1.set_xlim(0, 3600)

# ECDF plot
sns.ecdfplot(data=Test, x="start", ax=ax2, hue="Brain region", palette=Config.color_dict)
ax2.set_title("ECDF Plot")

ax2.set_xlim(0, 3600)


plt.tight_layout()
plt.show()

# Apply to whole dataset

In [ ]:
importlib.reload(Config)

In [ ]:
# Initialize random generator
rg = np.random.default_rng()

# Process brain regions
miscellaneous_nicknames = []
for brain_region in registries["brain_regions"]:
    output_path = f"/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250120_KDE_ECDF/KDE_ECDF_plots_{brain_region}.png"
    
    if os.path.exists(output_path):
        print(f"Skipping Brain region {brain_region} as the plot already exists.")
        continue
    
    region_data = Transformed[Transformed['Brain region'] == brain_region]
    nicknames = region_data['Nickname'].unique()
    
    if len(nicknames) < 5 and brain_region != "Control":
        miscellaneous_nicknames.extend(nicknames)
        continue
    
    try:
        Config.create_and_save_kde_ecdf_plot(Transformed, nicknames, brain_region, output_path, registries)
        print(f"Processed Brain region {brain_region}")
    except Exception as e:
        print(f"Error processing Brain region {brain_region}: {e}")

# Process Miscellaneous brain region
if miscellaneous_nicknames:
    try:
        output_path = "/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250120_KDE_ECDF/KDE_ECDF_plots_Miscellaneous.png"
        Config.create_and_save_kde_ecdf_plot(Transformed, miscellaneous_nicknames, "Miscellaneous", output_path, registries)
        print("Processed Miscellaneous Brain region")
    except Exception as e:
        print(f"Error processing Miscellaneous Brain region: {e}")

# Plot the Control brain region
try:
    control_nicknames = registries["control_nicknames"]
    output_path = "/mnt/upramdya_data/MD/MultiMazeRecorder/Plots/250120_KDE_ECDF/KDE_ECDF_plots_Control.png"
    Config.create_and_save_kde_ecdf_plot(Transformed, control_nicknames, "Control", output_path, registries)
    print("Processed Control Brain region")
except Exception as e:
    print(f"Error processing Control Brain region: {e}")